# Imports

In [ ]:
import proplot as plot
import climpy
import numpy as np
import numpy.fft as fft
import scipy.signal as signal
import matplotlib.ticker as mticker
plot.nbsetup()

In [ ]:
f['t'].dims

In [ ]:
f['u'].sum(dim=('time','plev','lat'))

In [ ]:
del f['t']

In [ ]:
f = climpy.xr.open_dataset('/Users/ldavis/data/eraint/climate1980-2015.nc', decode_times=False).squeeze()
f['t']

In [ ]:
climpy.deriv1_uneven(1, [1, 2, 3], keepedges=False, axis=0)

# EOF R^2 proof
This proves that the R^2 values obtained from projecting the PC onto each gridpoint, when summed across the grid, exactly equal
the eigenvalue (i.e. the variance explained).

In [ ]:
data = (np.random.rand(100,5) - 0.5).cumsum(axis=0)
data = data - data.mean(axis=0, keepdims=True)
var = data.var(axis=0)

In [ ]:
evals, nstar, proj, pcs = climpy.eof(data, record=0, space=1)

In [ ]:
evals
covar = (data.T @ data) / data.shape[0]
total = np.trace(covar)
evars = (evals*total/100)
# evar, evar.sum()

In [ ]:
evars.sum(), var.sum(), total

In [ ]:
# pc = pcs[0,:]
for evar,pc in zip(evars,pcs):
    data_a = data - data.mean(axis=0, keepdims=True)
    rsquare = ((data_a*pc).sum(axis=0))**2 / ((data_a**2).sum(axis=0) * (pc**2).sum(axis=0))
    print(evar, (var*rsquare).sum())

# Libby

In [ ]:
%run ./climpy/libby/butterworth_lanczos_example.py

In [ ]:
%run ./climpy/libby/testing_degrees_of_freedom_factors.py

In [ ]:
%run ./climpy/libby/testing_spectral_significance.py

In [ ]:
%run ./climpy/libby/seasonal_cycle_filtering_FFT.py

# Red noise and trends

In [ ]:
d = climpy.rednoise(500, 0.98, init=[-3,0,3], samples=3)
# d = climpy.rednoise(500, 0.99, init=0, samples=[3,3])
r = climpy.rolling(d, 50, axis=0, fillvalue=np.nan)
s = climpy.linefit(d, axis=0, build=True)
fit = climpy.linefit(d, axis=0, stderr=True)
print(fit)
# l = climpy.lanczos(30)
f, ax = plot.subplots()
for i in range(d.shape[1]):
    color = f'C{i}'
    h = ax.plot(d[:,i], color=color)
    h = ax.plot(r[:,i], color=color, alpha=.5, ls='--')
    h = ax.plot(s[:,i], color=color, alpha=.2, lw=2)
ax.format(xlabel='x', ylabel='y', title='Red noise with window and line fit')

# EOFs
In future, need to supply data that actually has *memory* or some kind of feedback.

## 1D
First try it out on a simple time series, for proof of concept. 

In [ ]:
import climpy
import numpy as np
import proplot as plot
plot.nbsetup()
def eofdata(nx, nt, alpha=0.95, scale1=1, scale2=1): # scales can be scalar, or space vectors
    # Get artificial EOF data. Note scale1/scale2 ***must be compatible shape***. You can make them 3D, singleton
    # 2-righthand dimensions, and that simulates creating another spatial dimension
    # Construct a see-saw of scale factors, varying from some number **smaller** than zero to **larger** than zero, and the 
    # **position** of the **center** of that see-saw (i.e. where factors equal 1) is a red noise time series.
    # Actually it didn't work and was dumb, nevermind
    # mid = climpy.rednoise(nt, 0.95, mean=np.pi/4, stdev=np.pi/24)
    # data = data*(1 + ((x[:,None] - mid[None,:]))) # so the scaling looks more linear
    
    # Just impose a random *phase* and *amplitude*.
    x = np.linspace(0, np.pi/2, nx)
    t = np.linspace(0, nt/4, nt)
    # scale1, scale2 = np.atleast_1d(scale1), np.atleast_1d(scale2)
    phase = climpy.rednoise(nt, alpha, mean=0, stdev=np.pi/12)
    amplitude = climpy.rednoise(nt, alpha, mean=1, stdev=0.2)
    data = scale2*amplitude[None,:]*np.sin(scale1*phase[None,:] + (x*2)[:,None])**2 # big scale equals strong phase effect
    return x, t, data

In [ ]:
import numpy as np
import climpy
import proplot as plot
plot.nbsetup()
# Coordinates
# Note shape will be x by time
t, x, data = eofdata(500, 500, alpha=0.98) # 500 x, 500 times

# Next get the EOFs
evals, nstar, projs, pcs = climpy.eof(data, record=-1, space=[-2], neof=5)
print('Data', data.shape)
print('Evals', evals.shape, 'Nstar', nstar.shape, 'Projection', projs.shape, 'PCs', pcs.shape)

# Plot data
f, ax = plot.subplots(axwidth=4, bottomcolorbar=True, innerpanels='r', innerpanels_kw={'wwidth':1}, aspect=1.3)
m = ax.contourf(x, t, data, cmap='sunset')
ax.format(xlabel='coordinate', ylabel='time', title='Time series')
res = f.bpanel.colorbar(m, clabel='magnitude')
h1, = ax.rpanel.plot(pcs[0,0,:], t, color='pink5', label='EOF1')
h2, = ax.rpanel.plot(pcs[1,0,:], t, color='yellow5', label='EOF2')
ax.rpanel.legend([h1, h2], entire=False, ncols=1, frameon=True, framealpha=0.8)
ax.rpanel.format(title='PC series')

# And plot them
f, ax = plot.subplots(axwidth=3)
h1, = ax.plot(x, projs[0,:,0], color='red7', linewidth=2, label='EOF1')
h2, = ax.plot(x, projs[1,:,0], color='blue7', linewidth=2, label='EOF2')
h2, = ax.plot(x, projs[2,:,0], color='gray5', linewidth=2, label='EOF3')
h3, = ax.plot(x, projs[3,:,0], color='gray5', linewidth=5, label='EOF4')
ax.axhline(0, color='k', ls='--', lw=2)
ax.format(xlabel='coordinate', ylabel='time', title='EOFs')
l = ax.legend(ncols=2)

## ND
We try getting EOF with *arbitrary* sample dimensions and *arbitrary* sampling dimension, then allow extra "N" dimensions.
Here, idea is we have **x by y by time by lat by lon** data, where the first 2 are random dummy "extra" dims.

In [ ]:
import numpy as np
import climpy
import proplot as plot
import scipy.stats as st
plot.nbsetup()
# New method, we just take the vector from above and tile it with varying scale factors
# Let's say the position EOF is strong on top, and the strength EOF is strongest on the bottom
nx, nt = 15, 100
ny = 15
# offset = 0 # will make lopsided scaling to one side
m1, m2 = 1.5, -1.5
scales1 = st.norm(m1, 1).pdf(np.linspace(-2, +2, ny)) # Gaussian curves
scales2 = st.norm(m2, 1).pdf(np.linspace(-2, +2, ny))
scales1 /= scales1.mean()
scales2 /= scales2.mean()
# scales1 = 2**(offset + np.linspace(-1, 1, ny)) # stronger on top
# scales2 = 2**(-offset + np.linspace(-1, 1, ny))
scales1 = scales1[:,None,None] # scale on an *extra dimension*
scales2 = scales2[:,None,None]

# Get data and scale it
x, t, data = eofdata(nx, nt, scale1=scales1, scale2=scales2)
y = x # the extra dimension; use same coordinates

# Get the EOFs
evals, nstar, projs, pcs = climpy.eof(data, record=-1, space=(-3,-2), neof=5, debug=True)
print('Data', data.shape)
print('Evals', evals.shape, 'Nstar', nstar.shape, 'Projection', projs.shape, 'PCs', pcs.shape)
f, axs = plot.subplots(innercolorbars='b', axwidth=3, ncols=2, span=0, share=0, wspace=0.5)
nlev = 11
data1 = projs[0,:,:,0].T
data2 = projs[1,:,:,0].T 
# data2 -= 10*projs[1,:,:,0].mean() # tests the 'zero' normalizer
m1 = axs[0].contourf(x, y, data1, cmap='Sunset', levels=nlev, extend='both')
m2 = axs[1].contourf(x, y, data2, cmap='NegPos', norm='zero', levels=nlev, extend='both')
axs[0].format(title='EOF1')
axs[1].format(title='EOF2')
axs.format(xlabel='x', ylabel='y')
res = axs[0].bottompanel.colorbar(m1)
res = axs[1].bottompanel.colorbar(m2)


In [ ]:
print(data.shape)
data2 = np.swapaxes(data, 0, 2)
data2 = data2[:,:,:,None]
print(data2.shape)
evals, nstar, projs, pcs = climpy.eof(data2, record=0, space=(1,2), neof=5, debug=True)
print(evals.shape, nstar.shape, projs.shape, pcs.shape)

In [ ]:
import xarray as xr
f = xr.Dataset({'u':(('lon','lat'),np.random.rand(5,5))})
for i in f.variables.keys():
    print(i)

In [ ]:
f = xr.open_dataset('/Users/ldavis/data/eraint/climate1980-2015.nc', chunks={'time':1}, decode_times=False)
f

In [ ]:
for name in f.keys():
    print(name)

In [ ]:
w = np.array([1, 1, 4])
x = np.array([0, 1, 2])
np.mean(x*w/w.mean())

In [ ]:
f, ax = plot.subplots(bottomcolorbar=True, axwidth=2.5)
n = 15
data = np.random.rand(n,n).cumsum(axis=0)
data
m = ax.contourf(data, levels=21, cmap='turquoise', extend='neither')
c = f.bpanel.colorbar(m)

# Spectral Filtering

## Lanczos and Butterworth

In [ ]:
x = np.linspace(0,20,1000) # 20 days, but with tons of data in-between
d = climpy.waves(x, wavelens=[1, 2])

In [ ]:
# Fake data
plot.globals('title', weight='bold')
plot.globals('suptitle', weight='bold')
n = 501
wsample = 10
cutoff = 1.2
cutoff2 = 0.5
waves = [0.3, 0.7, 2]
x = np.linspace(0,wsample,n) # 20 days, but with tons of data in-between
data = climpy.waves(x, np.array(waves), phase=None)
# Iterate
filters = ['lanczos', 'butterworth']
# filters = ['butterworth', 'lanczos']
# filters = ['butterworth']
for filt in filters:
    # Create filters
    dx = x[1]-x[0]
    if filt=='lanczos':
        wfilter = 2
        w = climpy.lanczos(dx, wfilter, cutoff)
        w2 = climpy.lanczos(dx, wfilter, cutoff2)
        suptitle = f'{wfilter}-day Lanczos filter'
        # suptitle = f'{len(w[0])}-day Lanczos filter'
        # w, w2 = [w], [w2]
    elif filt=='butterworth':
        wfilter = 11 # should be odd-numbered
        w = climpy.butterworth(dx, wfilter, cutoff)
        w2 = climpy.butterworth(dx, wfilter, cutoff2)
        suptitle =  f'order-{len(w[1])} Butterworth filter'
    colors = ('red5', 'blue5')
    nf = 2 if filt=='butterworth' else 1 # in *this case*, for display purposes, need to prevent shifting to left or right
    # Preparation for drawing
    radians = False
    scale = 2*np.pi if radians else 1
    cutoffs = (cutoff,cutoff2)
    weights = (w,w2)
    # Draw stuff
    f, axs = plot.subplots(right=0.2, left=0.7, top=0.5, array=[[1],[0],[3],[4]], hratios=(1,-.25,1,1),
                           sharex=False, spany=False, hspace=.7, aspect=2)
    ax = axs[1]
    for ic,iw,color in zip(cutoffs,weights,colors):
        # s, r = climpy.lanczos(width, ic, dx, response=True)
        s, r = climpy.response(dx, *iw) # b and a coefficients, or maybe just b
        # print(s.max()), print(x.max())
        s = s*scale # optionally convert to radians
        r = r.copy()
        h = ax.plot(s, r, color=color, lw=2)
        ax.axvline(scale/ic, color='k', ls='--', lw=2, alpha=0.5) # the cutoff wavelenghs, converted to wavenumber
    xlim = [1e-1, 5]
    if not radians: # wavenumbers on x-axis
        xlocator = 1 # frequencies of interest
        xlabel = 'wavenumber (day$^{-1}$)'
        xformatter = None
    else: # frequency in radians on x-axis
        xlim[1] *= np.pi
        xlocator = plot.arange(0,np.pi*8,np.pi*0.5) # frequencies of interest
        xlabel = 'frequency (rad day$^{-1}$)'
        xformatter=plot.PiFormatter()
    ax.format(xscale='linear', xlim=xlim, xlocator=xlocator, xtickminor=False,
              xformatter=xformatter, ylim=(-.1,1.1),
              xlabel=xlabel, ylabel='Window coefficients') # frequency i.e. radians per time unit
    # xlocator = np.array([0.1, 0.5, 1, 5, 10])
    xlocator = np.array([0.1, 0.5, 1, 5])
    ax2 = ax.twinx()
    ax2.format(xscale='inverse', xlim=xlim,
               xlabel='wavelength (day)', xlocator=xlocator, title='Response function',
               xtickdir='in', # xticklabeldir='in',
               xtickminor=True, xgrid=True, xgridminor=False)
    ax2.xaxis.grid(False, which='major')
    ax2.title.update({'position':(0.5,1.1)})
    for tick in ax2.xaxis.majorTicks:
        tick.gridline.set_visible(False)
    # Impulse response
    ax = axs[0]
    idata = data.copy()
    idata[:] = 0
    idata[0] = 1
    idata[len(idata)//2] = 1
    ifilter = climpy.filter(idata, *w, n=nf, axis=0, fix=False)
    ifilter2 = climpy.filter(idata, *w2, n=nf, axis=0, fix=False)
    ax.plot(x, idata, color='k', label='raw', alpha=0.8)
    ax.plot(x, ifilter, color=colors[0], alpha=0.8, ls='-', lw=2, label='lowpass 1')
    ax.plot(x, ifilter2, color=colors[1], alpha=0.8, ls='-', lw=2, label='lowpass 2')
    ax.legend()
    ylim = max(np.nanmax(np.abs(ifilter)), np.abs(np.nanmax(ifilter2)))*1.1
    ax.format(xlim=(0,x.max()), suptitle=suptitle,
              xlabel='x (day)', ylabel='response', title='Impulse response', ylim=(-ylim, ylim))
    # Next play with sample data
    # Can show that, given some weights, lfilter does exact same thing as rolling() function
    # lanczos_roll = climpy.rolling(data, w, axis=0)
    # lanczos_roll2 = climpy.rolling(data, w2, axis=0)
    ax = axs[2]
    lfilter = climpy.filter(data, *w, n=nf, axis=0) # with builtin signal method
    lfilter2 = climpy.filter(data, *w2, n=nf, axis=0)
    ax.plot(x, data, color='gray5', label='raw', alpha=0.8)
    # ax.plot(x, lanczos_roll, color='r', alpha=1, ls='--', lw=2, label='Lanczos')
    # ax.plot(x, data-lanczos_roll2, color='orange', alpha=0.2, ls='-', lw=2, label='Lanczos')
    # ax.plot(x, lanczos_roll2-lanczos_roll, color='indigo5', alpha=1, ls='-', lw=2) # bandpass attempt
    ax.plot(x, lfilter, color='r', alpha=0.8, ls='-', lw=2, label='lowpass') # capture low-freq oscillation
    ax.plot(x, data - lfilter2, color='orange', alpha=0.2, ls='-', lw=2, label='highpass') # capture high-freq oscillation
    ax.plot(x, lfilter2 - lfilter, color='indigo5', alpha=0.8, ls='-', lw=2, label='bandpass') # capture middle oscillation
    ax.format(xlabel='x (day)', ylabel='y', title='Sample data',
              # ylim=(-.01,.01), yformatter=plot.Formatter(precision=3),
             )
    ax.legend(ncols=4)
    f.save(f'{filt}_display.pdf')

## Coefficients

In [ ]:
# Copied from above, use this to plot coefficients -- not really needed because a-coeffs tell us nothing and the non-recursive
# b coefficients show up *exactly* (not approximately) in an impulse response plot
# Dummy plot
ax = axs[0]
labels = [0,0]
for ic,iw in zip(colors,weights):
    for j,jw,hue,coef in zip((0,1),iw,('gray','red'),('b','a')):
        if not (hasattr(jw,'__iter__') and len(jw)>0):
            jw = [np.nan]
        nl = len(jw) # length of window
        xw = np.arange(-nl//2+1, nl//2+1)
        if not labels[j]:
            label = coef
            labels[j] = 1
        else:
            label = ''
        ax.plot(xw, jw, color=f'{hue}{ic}', label=label)
ax.format(ylabel='coefficient (a or b)', xlabel='lag', title='Filter',
          suptitle=f'order-{nl:d} {filt.title()} filter')
ax.legend()

## StackOverflow

In [ ]:
# Example copied from: https://stackoverflow.com/a/12233959/4970632
from scipy.signal import butter, lfilter
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import freqz

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    print('cutoffs in radians:', low,high)
    b, a = butter(order, [low, high], btype='band')
    print('coefficients:', b,a)
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 5000.0 # sample rate i.e. seconds/timestep; cutoffs are in radians, convert by dividing by 2*sample_rate
lowcut = 500.0 # in Hz apparently
highcut = 1250.0

# Plot the frequency response for a few different orders.
plt.figure(1)
plt.clf()
for order in [3, 6, 9]:
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    w, h = freqz(b, a, worN=2000)
    plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
         '--', label='sqrt(0.5)')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.grid(True)
plt.legend(loc='best')

# Filter a noisy signal.
T = 0.05
nsamples = T * fs
t = np.linspace(0, T, nsamples, endpoint=False)
a = 0.02
f0 = 600.0
x = 0.1 * np.sin(2 * np.pi * 1.2 * np.sqrt(t))
x += 0.01 * np.cos(2 * np.pi * 312 * t + 0.1)
x += a * np.cos(2 * np.pi * f0 * t + .11) # seek this one
x += 0.03 * np.cos(2 * np.pi * 2000 * t)
plt.figure(2)
plt.clf()
plt.plot(t, x, label='Noisy signal')

y = butter_bandpass_filter(x, lowcut, highcut, fs, order=6)
plt.plot(t, y, label='Filtered signal (%g Hz)' % f0)
plt.xlabel('time (seconds)')
plt.hlines([-a, a], 0, T, linestyles='--')
plt.grid(True)
plt.axis('tight')
plt.legend(loc='upper left')
plt.show()

## SciPy Docs

In [ ]:
# Copied from scipy documentation
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
b, a = signal.butter(4, 100, 'low', analog=True)
w, h = signal.freqs(b, a)
plt.plot(w, 20 * np.log10(abs(h)))
plt.xscale('log')
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
# plt.figure()
# plt.plot(x, data)
# plt.plot(x, signal.lfilter(b, a, data))

# Power spectra

## 1-D

In [ ]:
x = np.linspace(0,100,10000) # 20 days
dx = x[1]-x[0]
# Data
# waves = [0.1, 0.2, 0.4, 0.6, 0.8, 3, 4, 5, 10, 30]
waves = [0.5, 1, 4]
window = len(x)//3 # 3 windows, or *5* overlapping windows
data = climpy.waves(x, waves, phase=None)

# Spectrum
# freq, power = climpy.power(data, dx, wintype=('gaussian',2000))
# freq, power = climpy.power(data, dx, wintype='boxcar', nperseg=2000)
freq, power = climpy.power(data, dx=dx, cyclic=False, manual=True, wintype='hann', nperseg=2000, scaling='spectrum')
freq = 1/freq # to wavelengths

# Figure
f, axs = plot.subplots(nrows=2, aspect=2, hspace=0.8, width=4, sharex=False, spany=False, hratios=(1,.5))
ax = axs[0]
ax.plot(x, data, label='raw')
ax.format(xlabel='x', ylabel='y', suptitle='Power spectra')
ax = axs[1]

# Plot
wnums = np.array([10, 0.3])
ax.plot(freq, power, label='power spectrum')
ax.format(xlim=1/wnums, xlabel='wavelength (days)', ylabel='strength')
var = data.var()
ax.text(-0.05, 1.5, f'total variance: {var:.1f}', va='top', weight='bold', transform='axes')
ax = ax.twiny()
ax.format(xlim=wnums[::-1], xscale='inverse', xlabel='wavenum (1/days)')
# ax.format(xlabel='wavelength (days)', ylabel='power (dB)', xscale='log', ylim=(-100,0))

## 2-D

In [ ]:
# Data
n2 = 1800
n1 = int(n2*0.25)
n1 = int(n2*0.5)
nperseg = 600
x1 = np.linspace(0,5,n1) # cyclic dim
x2 = np.linspace(0,5,n2) # non-cyclic dims
offset = np.linspace(0,1.5*np.pi,n2)[::-1]
w1 = [2]
w2 = [5]
d1 = climpy.waves(x1[:,None] + offset[None,:], w1)
d2 = climpy.waves(x2[None,:], w2) # changing phase as we go up
dx1 = x1[1]-x1[0]
dx2 = x2[1]-x2[0]
data = d1 + d2

In [ ]:
# Note: -2 transform will be transform of *real* data (i.e. symmetric), so left-half taken, but -1 transform
# will be transform of *complex* data, so both halves remain
f1, f2, result = climpy.power2d(data, dx=dx1, dy=dx2, nperseg=nperseg, axes=(0,1))
fig, axs = plot.subplots(nrows=2, aspect=2, width=5, sharex=False, spany=False, bottomcolorbar=True)
# result = 10*np.log10(result)
ax = axs[0]
ax.contourf(x1, x2, data, cmap='sunset')
ax.format(suptitle='2-D Transform, ClimPy', xlabel='x', ylabel='y')
ax = axs[1]
m = ax.contourf(f1, f2, result, cmap='sunset', levels=np.linspace(result.min(),result.max(),11))
xl = 6
ylim = (0, 6)
ax.format(xlabel='x-wavenumber', ylabel='y-wavenumber', xlim=(-xl,xl), ylim=ylim)
fig.bottompanel.colorbar(m, clabel='power (dB)')

In [ ]:
# Note: -2 transform will be transform of *real* data (i.e. symmetric), so left-half taken, but -1 transform
# will be transform of *complex* data, so both halves remain
result = fft.rfft2(data, axes=(1,0)) # right-hand uses symmetric transform, left-hand not
f2 = fft.fftfreq(data.shape[0])
f1 = fft.rfftfreq(data.shape[1])
result = np.abs(result)**2 # the power spectrum! that's it!
f, axs = plot.subplots(nrows=2, aspect=2, sharex=False, spany=False)
ax = axs[0]
ax.contourf(x1, x2, data, cmap='hclBlue')
ax.format(suptitle='2-D Transform')
ax = axs[1]
ax.contourf(f1, f2, result.T, cmap='hclBlue', levels=np.linspace(0,result.max(),11))
xval = 0.005
yval = 0.01
formatter = plot.Formatter(precision=4)
ax.format(xlim=(-xval,xval), ylim=(0,yval), xformatter=formatter, yformatter=formatter)